In [1]:
from brian2 import *
import numpy as np
import pickle

In [2]:
s = np.genfromtxt('stimulus.csv', delimiter=',')


array([0.77770241, 0.77886997, 0.77764333, ..., 0.29560571, 0.29676322,
       0.29227373])

In [3]:
# network param.
N = 500;
N_E = N
N_readout = 400

n_trials = 30

# neuron param.
tau_mem = 20*ms
thresh = 20 *mV
V_reset = 10*mV
V_rest = 0*mV
#thresh = -55 *mV
#V_reset = -65*mV
#V_rest = -75*mV
tau_rp = 2*ms
D = 1.5 *ms # Delay

eqs = '''
dv/dt = -(v-V_rest)/tau_mem : volt (unless refractory)
'''

# synapse param.
J_E = 0.2*mV
J_EE = 10*J_E
J_I = J_EE
#nu_thr = thresh / (J_E * tau_mem)

theta_0 = np.linspace(0,1,N + 1)
theta_0 = theta_0[:-1] #remove last element (circular stimulus variable)

# preferred locations of readout layer neurons
readout_theta_0 = np.linspace(0,1,N_readout + 1)
readout_theta_0 = readout_theta_0[0:-1] #same as for theta0

max_input = 5000* Hz #Heuristically set

#controlls width of input modulation to input layer
w = 0.3 
# Spatial frequency of input layer neurons
f = 1 

# Poisson input rates to the input layer
# Baseline input is 0.85*5000 = 4250 just enough to make neurons spike a few times, rest 0.15*5000 = 750 is stimulus modulation
input_rates = lambda theta: (0.15*exp(1/w*(cos(2*pi*f*(theta-theta_0)) - 1)) + 0.85 )*max_input

In [4]:
#activity input to input layer
TC_vec = [list(input_rates(min(1,max(s[i],0)))) for i in range(len(s[:]))]
stimulus = TimedArray(TC_vec, dt=0.1*ms)

In [5]:
for k in range(0,n_trials):
    #baseline input activity to readout layer
    start_scope()
    defaultclock.dt = 0.1 * ms

    P = PoissonGroup(N, rates='stimulus(t, i)')
    MP = SpikeMonitor(P)
    run(1000*ms)
    # And keep a copy of those spikes
    spikes_i = MP.i
    spikes_t = MP.t
    start_scope()
    defaultclock.dt = 0.1 * ms
    P = PoissonGroup(N_readout, rates=0.85*max_input)
    MP = SpikeMonitor(P)
    run(1000*ms)
    # And keep a copy of those spikes
    spikes_i_readout_base = MP.i
    spikes_t_readout_base = MP.t
    #run network simulations
    start_scope()
    defaultclock.dt = 0.1 * ms
    neurons = NeuronGroup(N, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )
    readout_neurons = NeuronGroup(N_readout, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )
    # Random initial states
    neurons.v = 10*(1 + rand(N_E))*mV # initial value
    readout_neurons.v = 10*(1 + rand(N_readout))*mV # initial value


    #Synapses input activity -> input layer
    SGG = SpikeGeneratorGroup(N, spikes_i, spikes_t)
    external_syn = Synapses(SGG, neurons, on_pre='v += J_E')
    external_syn.connect(j='i') #connect 1-to-1

    #Synapses input layer -> readout layer (distance dependent EPSPs - in stimulus space)
    read_out_syn = Synapses(neurons, readout_neurons, 'epsp : volt', on_pre='v += epsp', delay = D)
    read_out_syn.connect()
    FWHM = 1/N_readout
    sigma = (FWHM/2) / sqrt(2*log(2))

    # Set the size of the difference in preferred orientation dependent EPSPs
    w_readout = (pi*FWHM)**2/(2*log(2))
    read_out_syn.epsp = 'exp(1/(w_readout)*(cos(2*pi*f*(1/N_E*i - 1/N_readout*j))-1))*J_EE'

    #Synapses baseline input to readout layer
    SGG_readout = SpikeGeneratorGroup(N_readout, spikes_i_readout_base, spikes_t_readout_base)
    external_syn_readout = Synapses(SGG_readout, target=readout_neurons, on_pre='v += J_E')
    external_syn_readout.connect(j='i') #connect 1-to-1

    #Winner take all style inhibitory synapses between readout layer (preferred-stimulus-distance dependent, "nearer" -> less inhibition)
    read_out_syn_inhib = Synapses(readout_neurons, readout_neurons, 'ipsp : volt', on_pre='v += ipsp', delay = D)
    read_out_syn_inhib.connect()
    read_out_syn_inhib.ipsp = '-abs(sin(pi*(1/N_readout*i-1/N_readout*j)))*J_I' #'-(1 - exp(1/0.15*(cos(2*pi*(1/N_readout*i-1/N_readout*j)) - 1)))*J_I'
    
    spike_monitor = SpikeMonitor(neurons)
    spike_monitor_readout = SpikeMonitor(readout_neurons)
    net = Network(neurons, readout_neurons, read_out_syn, SGG, external_syn, SGG_readout, external_syn_readout, read_out_syn_inhib, spike_monitor, spike_monitor_readout)
    net.run(1000*ms)
    t_idx = (1/defaultclock.dt)*spike_monitor_readout.t[:]
    t_idx = t_idx.astype(int)
    
    data_1 = spike_monitor.get_states(['t','i','count'])

    
    data_readout = spike_monitor_readout.get_states(['t','i','count'])
    with open('./data/Gaussian_population/data_TC_layer_{}.pickle'.format(k), 'wb') as save_file:
        pickle.dump(data_1, save_file)

    with open('./data/Gaussian_population/data_readout_layer_{}.pickle'.format(k), 'wb') as save_file:
        pickle.dump(data_readout, save_file) 

WARNING    Cannot use Cython, a test compilation failed: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/ (DistutilsPlatformError) [brian2.codegen.runtime.cython_rt.cython_rt.failed_compile_test]
INFO       Cannot use compiled code, falling back to the numpy code generation target. Note that this will likely be slower than using compiled code. Set the code generation to numpy manually to avoid this message:
prefs.codegen.target = "numpy" [brian2.devices.device.codegen_fallback]
